# Proposed format

References:
* book: https://www.ryerson.ca/mining-complex-networks
* notebooks: https://github.com/ftheberge/GraphMiningNotebooks
    
**Part 1: EDA**

- 1.1 Relational data as graphs
    - basic igraph commands, main dataset, types of graphs
- 1.2 Exploratory data analysis (EDA)
    - degree, paths, connected components
- 1.3 Node importance: 
    - coreness, centrality, betweenness

**Part 2: Clustering**

- 2.1 Random graph models and benchmarks (ex: Erdos-Renyi, ABCD)
- 2.2 Transitivity
    - triangles and other motifs, local and global clustering coefficients, cliques
- 2.3 Clustering: modularity, Louvain, Leiden, ECG and properties; other methods
    - illustrations on various graphs; measures of cluster quality; ego-nets vs clusters
- 2.4 Extra module: Anomaly detection
    - illustration with football dataset

Some **igraph** references:
* https://igraph.readthedocs.io/en/0.10.2/
* https://towardsdatascience.com/newbies-guide-to-python-igraph-4e51689c35b4


## Required packages

* Minimal conda environment

```
conda create --name shortcourse python=3.7.9 numpy pandas jupyter matplotlib pycairo scikit-learn
source activate shortcourse
pip install igraph
pip install partition-igraph

python -m ipykernel install --user --name=shortcourse ## add to Jupyter

```

* Caveat: igraph is NOT the best tool for graph visualization, there are more specialized tool for this such as Graphviz, Bokeh, etc. However, igraph is a very powerful tool for graph analysis, is scalable (C background) and has R, Python and now Mathematica interfaces. We will use viz mainly to illustrate some concepts.

In [ ]:
## path to the datasets
datadir='./Datasets/'

## required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import igraph as ig
import partition_igraph
from collections import Counter
from sklearn.metrics import adjusted_rand_score as ARI
from sklearn.metrics import adjusted_mutual_info_score as AMI


# Part 1 - Basic Concepts and Exploratory Data Analysis (EDA)


## 1.1 Relational data as graphs

We will use the open source US airport graph as our main dataset:

* 2008 air traffic in the USA
* 464 nodes representing airports
* 12,000 edges representing travel from airport A to airport B, weighted by number of passengers 
* Some extra node attributes (city, state, lat/lon)
* Open access data

Some initial questions to ask when looking at (pairwise) relational data:
* is there a sense of **direction** to the relationship? (directed vs undirected graphs)
    * ex: direction of travel from airport A to B, I follow you vs. we are friends

* are some ties **stronger** than others? This is usually modelled in one of two ways:
  * **weights**: higher weights can mean stronger ties (ex: number of passengers; number of common friends);
  * **distance**: smaller distance can mean stronger ties (ex: distance between airports; commute time between cities; resistance in an electric circuit)

* can there be a relationship with oneself? This is modelled by edges from a node back to **itself**. (loops vs simple graphs)
  * ex: XXX add one

* are there other **attributes**? This can either be of the nodes or the edges.
  * ex: city, state as a node attributes

In [ ]:
## read edges from csv file
D = pd.read_csv(datadir + 'Airports/connections.csv')
D.head() ## look at a few edges

### Build a weighted directed graph from the edges

One way to create a (directed) graph in igraph is by handing it a list of edges in tuples of the form `(source, target, weight)`. We'll convert the above dataframe to a list of tuples and create a directed, weighted graph from it. 

In [ ]:
tuple_list = [tuple(x) for x in D.values]
g = ig.Graph.TupleList(tuple_list, directed=True, edge_attrs=['weight'])

### Graph Objects in igraph

A graph in igraph consists of:
* a **vertex** sequence object, with 0-based indices
* an **edge** sequence object, each connecting a 2-ple of vertices

Note: nodes and vertices mean the same thing. igraph uses the language of vertices and we will use node and vertex interchangeably.

**WARNING**: node names, if stored, can also be integers and may not correspond to node indices.


### Vertices

Vertices can be accessed by index (NOT the same thing as accessing the vertex by the name)

In [ ]:
g.vs[0]

It's useful to be able to `.find()` a vertex given its name (be careful if names are also integers!)

In [ ]:
g.vs.find('LAX')

In [ ]:
g.vs.find('LAX').index

In [ ]:
[v for v in g.vs[:5]]

Any vertex attribute may be added. In this case, the vertex set is used as a dictionary where the keys are the
attribute names. The values corresponding to the keys are the values of the given attribute for every vertex selected by the sequence. 

In [ ]:
g.vs[0]['color'] = ['green']
[v for v in g.vs][:5]

If you specify a sequence that is shorter than the number of vertices in
vertex sequence, the sequence is reused:

In [ ]:
g.vs['color'] = ['lightblue', 'pink', 'purple']
[v for v in g.vs][:5]

It's easy to access the number of vertices, in this case, airports. 

In [ ]:
g.vcount()

### Edges

Edges are  accessed by index in the edge sequence

In [ ]:
g.es[0]

Edges have a tuple of vertex indices of an edge representing `(source, target)`

In [ ]:
g.es[0].tuple

In [ ]:
g.es[0].source, g.es[0].target

Let's look up the details of the edge. 

In [ ]:
e_idx = 0

source_idx = g.es[e_idx].tuple[0]
target_idx = g.es[e_idx].tuple[1]

source_name = g.vs[source_idx]['name']
target_name = g.vs[target_idx]['name']
edge_weight = g.es[e_idx]['weight']

print(source_name, '--->' ,target_name,'has weight',edge_weight)

In [ ]:
# package this in a function
def print_edge_details(g, e_idx):
    """Helper function taking a graph and edge index to display edge information"""
    print(g.vs[g.es[e_idx].tuple[0]]['name'], '--->',
          g.vs[g.es[e_idx].tuple[1]]['name'], 'has weight',g.es[e_idx]['weight'])

Is there an edge in the other direction? We can check by looking up an edge by vertex ids.

In [ ]:
rev_e_idx = g.get_eid(target_idx, source_idx)
rev_e_idx

In [ ]:
print_edge_details(g, rev_e_idx)

Number of edges

In [ ]:
g.ecount()

It's easy to check if there is a way to get from one airport to another. This is like asking if the nodes are **connected**, that is, is there a path of edges between them. 

In [ ]:
## sanity check if SFO and LAX are connected. We know this to be true already.
g.are_connected(0,1)

In [ ]:
# We'll see later that we can't get from BMI to SFO. 
g.vs.find('BMI').index

In [ ]:
g.are_connected(0, 100)

What if we want to find all routes orignating from SFO? We can do this by asking which edges are **incident**.

In [ ]:
## list of all edges a given vertex is incident on
len(g.incident(0, mode='out'))

There are a lot. Let's see what a few of them are. 

In [ ]:
n = 5
for e_idx in  g.incident(0, mode='out')[:n]:
    print_edge_details(g, e_idx)

### Attributes

A common **edge attribute** is the edge weight, or distance. 

For the airport dataset, we also have a few node attributes:
 * City and state
 * Latitude and longitude (useful for nice layout)

These attributes are in a separate file, let's add them to the graph.

In [ ]:
airport_df = pd.read_csv(datadir + 'Airports/airports_loc.csv')
airport_df.head()

We have attributes by airport code, which is our vertex names. We need to lookup the vertex indices to add the attributes. 

In [ ]:
lookup = {k:v for v,k in enumerate(airport_df['airport'])}
l = [lookup[x] for x in g.vs()['name']]

## example
v = 0
print('vertex',v,':',g.vs[v]['name'], 'is at row', l[v])
airport_df.loc[l[v]]

Let's add the attributes to the graph.

nb: we use layout = (longitude, -latitude) due to location of origin

In [ ]:
g.vs['layout'] = [(airport_df['lon'][i],-airport_df['lat'][i]) for i in l]
g.vs['state'] = [airport_df['state'][i] for i in l]
g.vs['city'] = [airport_df['city'][i] for i in l]

Let's look at one vertex



In [ ]:
g.vs[0]

### Subgraphs and different types of graphs
To make our next analysis easier, we'll work off of a small subgraph.

In [ ]:
subgraph_nodes = [g.vs.find(name='LAX').index,
                  g.vs.find(name='SFO').index,
                  g.vs.find(name='OAK').index]
subgraph_nodes

It's easy to get an induced subgraph from a list of nodes

In [ ]:
sg = g.subgraph(subgraph_nodes)

In [ ]:
ig.plot(sg, bbox=(250,250), vertex_label=sg.vs['name'], 
        vertex_label_size=8, margin=50)

What if we want to know how many people travelled between the airports? In this case we don't care about the direction. We can do this by creating an undirected weighted graph, where the new weight are the summed edge weights from before.

In [ ]:
undirected_sg = sg.as_undirected(combine_edges=sum)

In [ ]:
ig.plot(undirected_sg, bbox=(250,250), vertex_label=sg.vs['name'], 
        vertex_label_size=8, margin=50)


If we want to get rid of the loops, we can use `simplify`.

In [ ]:
sg = sg.simplify(combine_edges=sum)

In [ ]:
ig.plot(sg, bbox=(250,250), vertex_label=sg.vs['name'], 
        vertex_label_size=8, margin=50)

Combining the above, we get a simple, undirected graph. 

In [ ]:
ig.plot(sg.as_undirected(combine_edges=sum),bbox=(250,250), vertex_label=sg.vs['name'], 
        vertex_label_size=8, margin=50)


The operations of `simplify` and `as_undirected` are non-descructive. But we redefined `sg` as the simple version above. `sg` is now a simple, directed graph. 

In [ ]:
ig.plot(sg, bbox=(250,250), vertex_label=sg.vs['name'], 
        vertex_label_size=8, margin=50)

We've been carrying the weights through, so let's visualize them by setting an edge width attribute. 

In [ ]:
sg.es['width'] = [int(np.log10(x)+1) for x in sg.es['weight']]

In [ ]:
## Weighted digraph
ig.plot(sg, bbox=(250,250), vertex_label=sg.vs['name'], 
        vertex_label_size=8, margin=50)

### Plotting in igraph

* igraph uses **cairo** for plotting, along with a python interface such as **pycairo** or **cairocffi**.
* latest versions of igraph can also use **matplotlib** (see example below)
* graph can be exported as **networkx** format which can be used in **bokeh** for interactive plotting
* another options is to save the graph in DOT format used in **GraphViz**


Here's an example of how to multiplot with matplotlib

In [ ]:
fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2, figsize=(8,8))
sg = g.subgraph(subgraph_nodes)
ig.plot(sg.as_undirected(combine_edges=sum), bbox=(250,250), vertex_label=sg.vs['name'], 
        vertex_label_size=8, margin=50, target=ax1)
sg = sg.simplify(combine_edges=sum)
ig.plot(sg.as_undirected(combine_edges=sum),bbox=(250,250), vertex_label=sg.vs['name'], 
        vertex_label_size=8, margin=50, target=ax2)
ig.plot(sg, bbox=(250,250), vertex_label=sg.vs['name'], 
        vertex_label_size=6, margin=50, target=ax3)
sg.es['width'] = [int(np.log10(x)+1) for x in sg.es['weight']]
ig.plot(sg, bbox=(250,250), vertex_label=sg.vs['name'], 
        vertex_label_size=6, margin=50, target=ax4);

## Other Representations of Graphs

Here's how to get from other representation of graphs from our igraph object:
* graph as a list of edges
* graph as an adjacency matrix
* graph as a weighted adjacency matrix

In [ ]:
df = pd.DataFrame()
df['from'] = [sg.vs[e.tuple[0]]['name'] for e in sg.es]
df['to'] = [sg.vs[e.tuple[1]]['name'] for e in sg.es]
df['volume'] = [e['weight'] for e in sg.es]
df.sort_values(by='volume', ascending=False, inplace=True)
df

In [ ]:
print(sg.get_adjacency())

In [ ]:
print(sg.get_adjacency(attribute='weight'))

### Questions
Try to answer the following questions using the graph objects that we've created.

#### 1. How many airports are in California (CA)?


#### 2. Which 5 states have the most airports? the least?
Hint: we've already imported `Counter` from `collections`

In [ ]:
# most


In [ ]:
# least


#### 3. Which airport is the southernmost? northernmost?

In [ ]:
# southernmost


In [ ]:
# northernmost 


#### 4. How many routes have at least 1 million passengers?



#### 5. Which route is the busiest one-way? both ways?


In [ ]:
# one-way


In [ ]:
# both ways


### Possible Solutions

In [ ]:
## airports in CA, three ways
print('airports in CA:', len([v for v in g.vs if v['state'] == 'CA']), 
      sum([v['state']=='CA' for v in g.vs]),
      sum(airport_df['state']=='CA'))

## states with most airports
print('\nmost airports:',Counter(g.vs['state']).most_common(5))

## states with the least airports
print('\nleast airports:',Counter(g.vs['state']).most_common()[-5:])

## north/south
latitude = [-x[1] for x in g.vs['layout']]
v = np.argmin(latitude)
print('\nsouthernmost:',g.vs[v],'\n')
v = np.argmax(latitude)
print('northernmost:',g.vs[v])

## 1M+ connections
print('\n1M+ connections:',len([e for e in g.es if e['weight'] >= 1000000]))

## busiest route (1-way) 
e = np.argmax(g.es['weight'])
print('\n1-way:',g.vs[g.es[e].tuple[0]]['name'], '---', g.es[e]['weight'], '-->', g.vs[g.es[e].tuple[1]]['name'], )

## busiest route (2-way) 
g_und = g.as_undirected(combine_edges=sum)
e = np.argmax(g_und.es['weight'])
print('\n2-way:', g_und.vs[g_und.es[e].tuple[0]]['name'], '---',  g_und.es[e]['weight'], '---', 
      g_und.vs[g_und.es[e].tuple[1]]['name'],)


## 1.2 Exploratory Data Analysis (EDA)

### Discussion

What makes network/graph data challenging?

* The points are connected and can not be considered as independent samples
* Inference on graphs requires the topological structure: node, edge, neighbourhoods, etc.
* The node's topological roles can be highly variable (degree, betweenness, centrality, etc.)


### Visualization

Let's take a look at the entire graph. 

First, using a **force directed layout**, we see a small **disconnected component**, this is not uncommon; in social networks and many other types of graphs, a common situation is to have a **giant component** with most nodes, and some small components.

In [ ]:
# reset the color of the vertices to all be the same
g.vs['color'] = 'lightblue'

ig.plot(g, vertex_size=5, edge_arrow_size=.5, edge_color='grey', 
        layout=g.layout_fruchterman_reingold(), bbox=(500,400))

Next let's use the latitutde and longitude based layout...this reveals a familiar shape.

In [ ]:
ig.plot(g, vertex_size=5, edge_arrow_size=.5, edge_color='grey', 
        layout=g.vs['layout'], bbox=(500,400))

This is a LOT to look at. Let's look at the subgraph induced by a single state. 

In [ ]:
st = 'MN'
sg = g.subgraph([v for v in g.vs if v['state'] == st])
ig.plot(sg,bbox=(450,350), vertex_label=sg.vs['name'], vertex_size=15,layout=sg.vs['layout'],
            vertex_label_size=6, margin=50)

### Node degree(s)

A key node feature is their **degree**.

With a **directed graph**, we can distinguish 4 concepts of degree (!)

* in-degree: number of edges coming in
* out-degree: number of edges going out
* total degree: sum of the above 2 quantities
* (undirected) degree: degree when reducing to **undirected** graph

Let's look at all four degrees in our airport graph.


In [ ]:
d = pd.DataFrame()
d['node'] = g.vs['name']
d['in-deg'] = g.degree(mode='in')
d['out-deg'] = g.degree(mode='out')
d['total-deg'] = g.degree(mode='all')
d['und-deg'] = g.as_undirected().degree()

In [ ]:
d.loc[d['node'].isin(['SFO','LAX','OAK','OPF'])]

### Ego-net of a node

This is subgraph induced by a node and its neighbours. Let's take a look the ego-net of OPF, the Miami executive airport.



In [ ]:
sg = g.induced_subgraph(g.neighborhood(g.vs.find(name='OPF')))
ig.plot(sg, bbox=(300,300), vertex_label=sg.vs['name'], 
            vertex_label_size=8, margin=50)

Here's the undirected version

In [ ]:
sg = sg.as_undirected()
ig.plot(sg, bbox=(300,300), vertex_label=sg.vs['name'], 
            vertex_label_size=8, vertex_color='lightblue', margin=50)

### Weighted degree a.k.a. Strength

With **weighted** graphs, we also define:

* in-strength: sum of weights of all incoming edges
* out-strength: sum of weights of all outgoing edges
* total-strength: sum of the above two quantities

When converting a directed graph to undirected, it is common to add the edge weights, so we get the "total-strength".


In [ ]:
d['in-str'] = g.strength(mode='in', weights='weight')
d['out-str'] = g.strength(mode='out', weights='weight')
d['total-str'] = g.strength(mode='all', weights='weight')
## undirected graph, summing the edge weights
d['und-str'] = g.as_undirected(combine_edges=sum).strength(weights='weight')
d.loc[d['node'].isin(['SFO','LAX','OAK','OPF'])]

### Degree distribution

Also common in most social-type networks, degree distribution is far from uniform in the airport graph, with lots of low degree nodes, and a small number of high degree ones.

This is indicative of **power-law** degree distribution.

In such networks, shortest paths between connected nodes are typically short (the **6-degree of separation** phenomenon).

In [ ]:
plt.figure(figsize=(8,3))
plt.subplot(121)
plt.hist(g.degree(mode='in'), bins=30)
plt.title('in-degree')
plt.xlabel('degree')
plt.ylabel('frequency')
plt.subplot(122)
plt.hist(g.degree(mode='out'), bins=30)
plt.title('out-degree')
plt.xlabel('degree');

### Paths and connected components

* A **path** is a sequence of edges connecting two nodes
* In a **connected component**, there is a path between every pair of nodes
* Path length is usually the number of edges (**hop count**), but weights can also be considered
* For directed graphs, we can take directionality into account (**strong** connectivity) or not (**weak** connectivity); those are the same in undirected graphs


In [ ]:
print('strong connectivity:', g.connected_components(mode='strong').summary())

In [ ]:
print('weak connectivity:',g.connected_components(mode='weak').summary())

In [ ]:
g_undirected = g.as_undirected(combine_edges=sum) ## undirected graph, summing the weights

In [ ]:
print('undirected connectivity:',g_und.connected_components().summary())

It can be interesting to consider where there are paths between nodes, and where there are not. Let's look at where we can get to starting from a given airport. Recall that there are 464 airports (number of nodes `.vcount()`).

Let's look at OPF and SFO.

In [ ]:
## shortest (directed) paths
for ap in ["OPF","SFO"]:
    print("\nlooking at:", ap)
    v = g.vs.find(name=ap)
    sp = g.distances(source=v, mode='out')[0]
    print('number of "unreacheable" airports:',sum([i == np.inf for i in sp]))
    print('mean number of hops to other airports:',np.mean([i for i in sp if i != np.inf ]))
    print('max number of hops to other airports:',np.max([i for i in sp if i != np.inf ]))


###  Why are some airports "unreacheable"?

Two airports are in a separate (weak) connected component. What about the other 18? Let's see if we can identify why all of these airports are unreachable.

In [ ]:
## color unreacheable nodes from SFO in red
unreachable = np.where([i == np.inf for i in sp])
for j in unreachable:
    g.vs[j]['color'] = 'red' 
unreachable

In [ ]:
ig.plot(g, vertex_size=5, edge_arrow_size=.5, edge_color='grey', 
        layout=g.layout_fruchterman_reingold(), bbox=(500,400))

We see 2 airports in small connected component 

In [ ]:
g.vs['cc'] = g.connected_components('weak').membership
Counter(g.vs['cc'])

Let's remove those 2 nodes. 

In [ ]:
v_list = [v['name'] for v in g.vs if v['cc']==1]
g_copy = g.copy()
g_copy.delete_vertices(v_list)

In [ ]:
ig.plot(g_copy, vertex_size=5, edge_arrow_size=.5, edge_color='grey', 
        layout=g_copy.layout_fruchterman_reingold(), bbox=(500,400))

Now let's also remove airports that have no incoming links.

In [ ]:
v_list = [v for v in g_copy.vs if g_copy.degree(v,'in') == 0]
print('removing',len(v_list),'more')
g_copy.delete_vertices(v_list)

There's still one red node left.

In [ ]:
ig.plot(g_copy, vertex_size=5, edge_arrow_size=.5, edge_color='grey', 
        layout=g_copy.layout_fruchterman_reingold(), bbox=(500,400))

Let's recursively remove airports without incoming links

In [ ]:
v_list = [v for v in g_copy.vs if g_copy.degree(v,'in') == 0]
print('removing', len(v_list),'more')
g_copy.delete_vertices(v_list)

That seems to have got them all. Once we remove the other component and recursively remove airports with no incoming links, all the airports become reachable from SFO. 

In [ ]:
ig.plot(g_copy, vertex_size=5, edge_arrow_size=.5, edge_color='grey', 
        layout=g_copy.layout_fruchterman_reingold(), bbox=(500,400))

### Questions

#### 1. Which airport has the largest number of outgoing connections? incoming? total?

In [ ]:
# outgoing


In [ ]:
# incoming


In [ ]:
# total


#### 2. Which airport has the largest number of incoming/outgoing/total passengers? the smallest?



In [ ]:
# incoming largest + smallest


In [ ]:
# outgoing largest + smallest


In [ ]:
# total largest + smallest


#### 3. What happens if we ignore direction of flights and consider: 
* the number of "unreacheable" airports from SFO, OPF
* the mean number of hops to other airports from SFO, OPF
* max number of hops to other airports from SFO, OPF

### Possible Solutions

In [ ]:

## outgoing connections
x = np.argwhere(g.degree(mode='out') == np.max(g.degree(mode='out'))).flatten()
for v in x:
    print(g.vs[v]['name'],"has outgoing connections to",g.degree(v,'out'),"airports")

## incoming connections
x = np.argwhere(g.degree(mode='in') == np.max(g.degree(mode='in'))).flatten()
for v in x:
    print(g.vs[v]['name'],"has incoming connections from",g.degree(v,'in'),"airports")

## any connections
x = np.argwhere(g_und.degree() == np.max(g_und.degree())).flatten()
for v in x:
    print(g_und.vs[v]['name'],"has connections with",g_und.degree(v),"airports")

## largest number of passengers - outgoing
v = np.argmax(g.strength(mode='out', weights='weight'))
print(g.vs[v]['name'],"has",g.strength(v,'out', weights='weight'),"outgoing passengers")

## largest number of passengers - incoming
v = np.argmax(g.strength(mode='in', weights='weight'))
print(g.vs[v]['name'],"has",g.strength(v,'in', weights='weight'),"incoming passengers")

## largest number of passengers - total
v = np.argmax(g_und.strength(weights='weight'))
print(g_und.vs[v]['name'],"has",g.strength(v, weights='weight'),"total passengers")

## smallest number of passengers - total
v = np.argmin(g_und.strength(weights='weight'))
print(g_und.vs[v]['name'],"has",g.strength(v, weights='weight'),"total passengers")

## undirected short paths
for ap in ["SFO","OPF"]:
    print("\nlooking at:",ap)
    v = g.vs.find(name=ap)
    sp = g.distances(source=v, mode='all')
    print('number of unreacheable airports:',sum([i == np.inf for i in sp[0]]))
    print('mean number of hops to other airports:',np.mean([i for i in sp[0] if i != np.inf ]))
    print('max number of hops to other airports:',np.max([i for i in sp[0] if i != np.inf ]))

## 1.3 Node importance


### Coreness (k-cores)

The **k-core** of a graph is the maximal subgraph where all nodes have degree at least k. The **coreness** of a node is k if it belongs to the k-core, but not the (k+1)-core. 

This is usually done with the **un-directed** degrees, but one can also look for 'in' and 'out' k-cores.

Below we compute coreness for the undirected version of the airport graph. We see:

* many nodes in large 50-core subgraph (which we plot)
* many nodes with small coreness (1 or 2)

Pruning nodes with small coreness often drastically reduces the size of the graph.

In [ ]:
g_und = g_und.simplify(combine_edges=sum)

## most frequent coreness values
g_und.vs['core'] = g_und.coreness()
Counter(g_und.vs['core']).most_common(3)

In [ ]:
plt.hist(g_und.vs['core'], bins=25);

Let's look at the largest k-core - plot with lat/lon layout

In [ ]:
sg = g_und.subgraph([v for v in g_und.vs if v['core'] == 50])
ig.plot(sg,bbox=(600,450), vertex_label=sg.vs['name'], vertex_size=15,layout=sg.vs['layout'],
            vertex_label_size=6, margin=50)

## Centrality, betweenness

There are various ways to define the most **important** nodes in a network.
Two families of measures are:

* Node **centrality**, which includes measures such as **degree** and **pagerank**.
 * those measure overall connectedness in various ways
* **Betweenness** measures the proportion of shortest paths going through some node or edge.

Let's explore this with the undirected airport graph.

In [ ]:
df = pd.DataFrame()
df['degree'] = g_und.degree()
df['pagerank'] = g_und.pagerank(weights='weight')
n = g_und.vcount()
df['between'] = [2 * x / ((n - 1) * (n - 2)) for x in g_und.betweenness()] ## normalized
df['state'] = g_und.vs['state']
df['city'] = g_und.vs['city']
df['name'] = g_und.vs['name']
df.sort_values(by='pagerank', inplace=True, ascending=False)
df.head()

In [ ]:
df.sort_values(by='between', inplace=True, ascending=False)
df.head()

In [ ]:
plt.scatter(df['degree'],df['pagerank'])
plt.xlabel('degree')
plt.ylabel('pagerank');

In [ ]:
plt.scatter(df['degree'],df['between'])
plt.xlabel('degree')
plt.ylabel('betweenness');

### Questions

There is one node with low degree but betweenness above 0.02.

#### 1. Which node is it?

#### 2. What could explain the low degree and relatively high betweenness score?



### Possible Solutions

In [ ]:
## low degree, high betweenness node
print(df.loc[ (df['degree']<25) & (df['between']>.02)])

## look at its ego-net ... along with edge weights
v = g_und.vs.find('LSV')
sg = g_und.subgraph(g_und.neighborhood(v))
ig.plot(sg,bbox=(600,400), vertex_label=sg.vs['name'], vertex_size=15,layout=sg.vs['layout'],
            vertex_label_size=6, margin=50, edge_label=sg.es['weight'])


# Bonus Exercise - a different graph

The airport graph, as well as most social type graphs, have some common structure: 
* **power-law** degree distribution
* **small world** behaviour (a.k.a. 6-degree of separation)
    
In the 'Example' subdirectory of the datasets, there are two files:
* edges: 2-ples, undirected, unweighted edges (no header)
* nodes: csv file with node attributes (with header)

Beware: node names are integers

Follow the steps:

#### 1. Build an undirected, unweighed graph using the 'edges' file and remove loops; how many nodes/edges are there?



#### 2. Plot a histogram of the degree distribution

#### 3. Find the mean/max path lengths from a few nodes to all nodes. From the results of the previous 2 steps ... what do you think this graph could be?

#### 4. Load the node attributes, which include lat/lon.  Use those to plot the graph, what now?

### Possible Solutions

In [ ]:
## 1. read edge list and build undirected simple graph
gr = ig.Graph.Read_Ncol(datadir+'Example/edges', directed=False, )
gr = gr.simplify()
print(gr.vcount(), 'nodes and',gr.ecount(),'edges')
print(gr.vs[0])

In [ ]:
## 2. degrees - mostly small values ...
print('max:',np.max(gr.degree()))
plt.hist(gr.degree(), bins=16);

In [ ]:
## 3. shortest paths for some a few node(s) -- much larger values than the airport graph ...
print('number of nodes:',gr.vcount())
for v in [0,1000]:
    print("\nlooking at node:",v)
    sp = gr.distances(source=v)
    print('number of unreacheable nodes:',sum([i == np.inf for i in sp[0]]))
    print('mean number of hops to other nodes:',np.mean([i for i in sp[0] if i != np.inf ]))
    print('max number of hops to other nodes:',np.max([i for i in sp[0] if i != np.inf ]))


In [ ]:
## 4. read node attributes
X = pd.read_csv(datadir+'Example/nodes')
X.head()

In [ ]:
## we have attribute by vertex name -- map to proper indices
lookup = {str(k):v for v,k in enumerate(X['name'])}
l = [lookup[x] for x in gr.vs()['name']]

## store layout attributes in graph and plot
## nb: we use negative latitude for layout due to location of origin
gr.vs['layout'] = [(X['lon'][i],-X['lat'][i]) for i in l]
ig.plot(gr, bbox=(500,400), layout = gr.vs['layout'], vertex_size=3, vertex_color='lightblue', margin=50)


### Europe Electric Grid

Network of high voltage grid in Europe. Vertices are stations and edges are the lines connecting them.
More details at: https://zenodo.org/record/47317#.Xt6nzy3MxTY


In [ ]:
## clean up
del(g.vs['cc'])
del(g_und.vs['core'])
del(gr)

# Part 2 - Clustering

## 2.1 Random Graph Models

Random graph models are useful for various reasons:
* interpretation of results on real graphs (ex: is this value expected? high? low?)
* to compare algorithms (ex: clustering algorithms)
* to study theoretical properties

Usually, we fix some aspects of the graph, such as the **number of nodes**, and randomly sample.
There are many such models, including:
* Erdos-Renyi model: fix the number of nodes and randomly place edges
* Chung-Lu/Configuration models: as above, but given a degree distribution for the nodes
* Barabasi-Albert model: "the righ gets richer" model; a new node is more likely to link with a hub
* LFR, ABCD models: power-law node degree distribution with ground-truth communities

Let's explore. 

First, an Erdos-Renyi (ER) graph with same number of nodes/edges as (undirected) airport graph

In [ ]:
g_er = ig.Graph.Erdos_Renyi(n=g_und.vcount(), m=g_und.ecount())
print('min degree',np.min(g_er.degree()),'max degree',np.max(g_er.degree()))
ig.plot(g_er, vertex_size=5, vertex_color='lightblue', edge_color='grey', bbox=(500,400))

Now a configuration model with the same degree sequence as airport graph



In [ ]:
g_ds = ig.Graph.Degree_Sequence(g_und.degree())
print('min degree',np.min(g_ds.degree()),'max degree',np.max(g_ds.degree()))
ig.plot(g_ds, vertex_size=5, vertex_color='lightblue', edge_color='grey', bbox=(500,400))

## 2.2 Triangles and Transitivity

* Edges represent relations between entities (nodes) in graphs/networks.

* The next fundamental motifs are **triangles**

* The presence of triangles is indicative of communities (dense subgraph(s) in a graph)

* Two fundamental measures of the presence of triangles in graphs are:

    * **transitivity** (global clustering coefficients) measures the proportion of wedges (two-hop path in an undirected graph) that form a triangle
    * **local transitivity** (local clustering coefficient) for a node is the proportion of pairs of neighbours that form a triangle; we also define the **average local transitivity** by averaging over all nodes of degree at least 2.

Those measures assume **undirected** graphs.

While we can define triangles and other motifs for directed graphs, clustering generally assumes undirected graphs.

We compute the above for the airport graph, as we look for nodes with high/low local transitivity.

In [ ]:
print('trans:',g_und.transitivity_undirected())
print('avg local trans:',g_und.transitivity_avglocal_undirected())

Now, local transitivity - nodes with degree < 2 will get 0

In [ ]:
g_und.vs['trans'] = g_und.transitivity_local_undirected(mode='zero') 

High transitivity

In [ ]:
v = np.argmax(g_und.vs['trans'])
print('airport:',g_und.vs[v]['name'],', plotting its ego-net:')
sg = g_und.subgraph(g_und.neighborhood(v))
ig.plot(sg,bbox=(400,300), vertex_label=sg.vs['name'], vertex_size=15,layout=sg.vs['layout'],
            vertex_label_size=6, vertex_color='lightblue', margin=50)


Transitivity for an ER graph



In [ ]:
print('trans:',g_er.transitivity_undirected())
print('avg local trans:',g_er.transitivity_avglocal_undirected())


Transitivity for random graph with the same degree sequence



In [ ]:
print('trans:',g_ds.transitivity_undirected())
print('avg local trans:',g_ds.transitivity_avglocal_undirected())


### Cliques

Triangles are also known as **3-cliques**; 
a clique of size k is a fully connected subgraph with k nodes.

The **clique number** is the size of the largest clique.


Cliques in the Airport graph

In [ ]:
print('number of 3-cliques:', len(g_und.cliques(min=3, max=3)))
print('number of 4-cliques:', len(g_und.cliques(min=4, max=4)))
print('max clique:', g_und.clique_number())

Cliques in the ER graph

In [ ]:
print('number of 3-cliques:', len(g_er.cliques(min=3, max=3)))
print('number of 4-cliques:', len(g_er.cliques(min=4, max=4)))
print('max clique:', g_er.clique_number())

Cliques in the degree sequence graph

In [ ]:
print('number of 3-cliques:',len(g_ds.cliques(min=3, max=3)))
print('number of 4-cliques:',len(g_ds.cliques(min=4, max=4)))
print('max clique:',g_ds.clique_number())

### Directed graphs

Clique finding assumes undirected graphs (see below).

For directed graphs, there are 16 different possibilities for each **triad**

REF: "The Structure of Positive Interpersonal Relations in Small Groups (1967)"

The **triad census** function counts all occurrences, and returns the counts in a specific ordering:
* '003': no edge
* '012': 1 directed edge only
* '102': 2 directed edges in opposite direction between the same pair of nodes
* ...
* '300': all 6 directed edges are present.


In [ ]:
len(g.cliques(min=3,max=3))

In [ ]:
tc = g.triad_census()
print('number of triads:',np.sum(tc))
print('by type:',tuple(tc))

### Questions

#### 1. Find the node in the undirected airport graph with degree 5 or more having the lowest transitivity.


#### 2. Which triad motif is the rarest in the (directed) airport graph? 



### Possible Solutions

In [ ]:
## node with low transitivity, degree at least 5
x = np.argmin([v['trans'] for v in g_und.vs if g_und.degree(v)>=5])
v = [v for v in g_und.vs if g_und.degree(v)>=5][x]
print('airport:',v['name'],', plotting its ego-net:')
sg = g_und.subgraph(g_und.neighborhood(v))
ig.plot(sg,bbox=(400,300), vertex_label=sg.vs['name'], vertex_size=15,layout=sg.vs['layout'],
            vertex_label_size=6, vertex_color='lightblue', margin=50)

## Recall that LSV is also the node we found to have low degree/centrality, but relatively high betweenness


In [ ]:
## The rarest triad motif is the 10th, which is motif '030C', a cycle of directed edges.

## There are more node pairs with two edges than with a single edge.
print('number of edges:',g.ecount())
g.es['mark'] = 1
print(Counter(g.as_undirected(combine_edges=sum).es['mark']))
del(g.es['mark'])

In [ ]:
## clean-up
del(g_und.vs['trans'])

## 2.3 Clustering

Graph clustering, a.k.a. node partitioning, is a very active research area, with dozens of algorithms.
Some good ones are:

* Louvain (multilevel): 
    * modularity optimization 
    * suffers from resolution limit (may merge "natural" communities)
    * may return disconnected communities
    * fast
    * somewhat unstable for graphs with homogeneous edge weights
* Leiden:
    * Louvain-based
    * faster
    * communities are all connected
* ECG (ensemble clustering):
    * Louvain-based
    * better stability for graphs with homogeneous edge weights
    * less affected by resolution limit
* Infomap
    * based on information theory concepts: encoding random walks is shorter within communities
    * good results with low to moderate noise graphs
    * may not converge at all for noisy graphs
* Label Propagation
    * very fast
    * will not converge at all for noisy graphs

Measures of community strength include:
* modularity ("proportion of edges. within communities" - "expected proportion under null model")
* comparing degree within and between communities overall
* comarting degree within and between communities for each vertex

Let's start by clustering the ER graph 

In [ ]:
ec = g_er.community_ecg()
ec.membership[:5] # component list of nodes

How many communities are there? 

In [ ]:
np.max(ec.membership) + 1 # membership is 0-indexed


What's the modularity?

In [ ]:
g_er.modularity(ec.membership)

Degree between and within communities

In [ ]:
g_er.es['label'] = 1
G = ec.cluster_graph(combine_edges=sum)
G.vs['label'] = [2*G.ecount() for G in ec.subgraphs()]
ig.plot(G, bbox=(400,300), vertex_label_size=8, edge_label_size=8)

### ABCD graphs to compare algorithms

This model generates graphs with power-law degree distribution and communities.

The strength of the communities is adjusted via a 'noise' parameter 'mu'.

Code is very fast (Julia): https://github.com/bkamins/ABCDGraphGenerator.jl

Even faster multi-thread version: https://github.com/tolcz/ABCDeGraphGenerator.jl

Generalizations in development:
* graphs with communities and outlier nodes
* hypergraphs

We'll look at two ABCD graphs, along with ground-truth communities:

* 1000 nodes, 14 communities, 1/3 'noise' edges (mu = 0.33)
* 1000 nodes, 14 communities, 2/3 'noise' edges (mu = 0.67)

### Comparing resulting communities

When ground truth is available, good measures to compare various clustering results are:
* Adjusted mutual information (AMI)
* Adjusted RAND index (ARI)
* Adjusted graph-aware RAND index (AGRI)

In [ ]:
def print_scores(g, clustering):
    print('Number of communities:', np.max(clustering.membership)+1)
    print('AMI:', AMI(g.vs['gt'],clustering.membership),
          '\tARI:', ARI(g.vs['gt'],clustering.membership),
          '\tAGRI:', partition_igraph.gam(g, {k:v['gt'] for k,v in enumerate(g.vs)}, clustering))
    print('modularity:', g.modularity(clustering.membership))

Let's build a graph where we know some ground truth about communities and evaluate different algorigthms. 

In [ ]:
mu = 33 ## 1/3 noise edges

In [ ]:
df = pd.read_csv(datadir+'ABCD/edges_'+str(mu)+'.dat', sep='\t', header=None)
abcd33 = ig.Graph.TupleList([tuple(x) for x in df.values], directed=False)

## load ground-truth communities - add as node attribute
df = pd.read_csv(datadir+'ABCD/communities_'+str(mu)+'.dat', sep='\t', header=None)
gt = dict([tuple(x) for x in df.values])
abcd33.vs['gt'] = [gt[x] for x in abcd33.vs['name']]

Here's how the different algorithms do using AMI, ARI, AGRI and modularity

In [ ]:
print('Number of ground-truth communities:', np.max(abcd33.vs['gt']),'\n')
print('Louvain:')
print_scores(abcd33, abcd33.community_multilevel())
print('\nLeiden:')
print_scores(abcd33, abcd33.community_leiden(objective_function='modularity'))
print('\nECG:')
print_scores(abcd33, abcd33.community_ecg())
print('\nInfomap:')
print_scores(abcd33, abcd33.community_infomap())
print('\nLabel Propagation:')
print_scores(abcd33, abcd33.community_label_propagation())

Let's visualize by coloring w.r.t. the ground truth communities

In [ ]:
nc = max(abcd33.vs['gt'])
pal = ig.ClusterColoringPalette(nc+1)
abcd33.vs['color'] = [pal.get(i) for i in abcd33.vs['gt']]
ly = abcd33.layout_fruchterman_reingold()
ig.plot(abcd33, layout=ly, vertex_size=5, bbox=(0,0,500,400), edge_color='lightgrey')

Let's collapse communities to show degree between and within communities

In [ ]:
ec = abcd33.community_ecg()
abcd33.es['label'] = 1
G = ec.cluster_graph(combine_edges={'label':sum})
G.vs['label'] = [2*G.ecount() for G in ec.subgraphs()]
del(abcd33.es['label']) ## clean up
ig.plot(G, bbox=(400,300), vertex_label_size=8, edge_label_size=8)

Let's look at what happens if we adjust the noise

In [ ]:
mu = 67 ## 2/3 noise edges

In [ ]:
## load edges, build graph
df = pd.read_csv(datadir+'ABCD/edges_'+str(mu)+'.dat', sep='\t', header=None)
abcd67 = ig.Graph.TupleList([tuple(x) for x in df.values], directed=False)

## load ground-truth communities - add as node attribute
df = pd.read_csv(datadir+'ABCD/communities_'+str(mu)+'.dat', sep='\t', header=None)
gt = dict([tuple(x) for x in df.values])
abcd67.vs['gt'] = [gt[x] for x in abcd67.vs['name']]

In [ ]:
print('Number of ground-truth communities:', np.max(abcd67.vs['gt']),'\n')
print('Louvain:')
print_scores(abcd67, abcd67.community_multilevel())
print('\nLeiden:')
print_scores(abcd67, abcd67.community_leiden(objective_function='modularity'))
print('\nECG:')
print_scores(abcd67, abcd67.community_ecg())
print('\nInfomap:')
print_scores(abcd67, abcd67.community_infomap())
print('\nLabel Propagation:')
print_scores(abcd67, abcd67.community_label_propagation())

In [ ]:
nc = max(abcd67.vs['gt'])
pal = ig.ClusterColoringPalette(nc+1)
abcd67.vs['color'] = [pal.get(i) for i in abcd67.vs['gt']]
ly = abcd67.layout_fruchterman_reingold()
ig.plot(abcd67, layout=ly, vertex_size=5, bbox=(0,0,500,400), edge_color='lightgrey')


In [ ]:
ec = abcd67.community_ecg()
abcd67.es['label'] = 1
G = ec.cluster_graph(combine_edges={'label':sum})
G.vs['label'] = [2*G.ecount() for G in ec.subgraphs()]
del(abcd67.es['label']) ## clean up
ig.plot(G, bbox=(400,300), vertex_label_size=8, edge_label_size=8)

### Clusters in the airport graph using Louvain

In [ ]:
## Louvain algorithm is randomized - fix the seed to compare results
import random
random.seed(31416)

In [ ]:
ml = g_und.community_multilevel(weights='weight')
g_und.vs['ml'] = ml.membership
pal = ig.ClusterColoringPalette(n=np.max(ml.membership)+1)
g_und.vs['color'] = [pal.get(i) for i in ml.membership]

In [ ]:
g_und.modularity(ml.membership, weights='weight')

In [ ]:
ig.plot(g_und, vertex_size=5, edge_color='grey', layout=g.vs['layout'], bbox=(500,400))

Collapse communities to show degree between and within communities

In [ ]:
g_und.es['label'] = 1
g_und.vs['lat'] = [v['layout'][0] for v in g_und.vs]
g_und.vs['lon'] = [v['layout'][1] for v in g_und.vs]
G = ml.cluster_graph(combine_vertices={'lat':np.mean,'lon':np.mean}, combine_edges={'label':sum})
G.vs['label'] = [2*G.ecount() for G in ml.subgraphs()]
ly = [(v['lat'],v['lon']) for v in G.vs]
ig.plot(G, layout=ly, bbox=(450,350), vertex_label_size=8, edge_label_size=8, vertex_color='lightblue')

In [ ]:
## clean up
del(g_und.es['label'])
del(g_und.vs['lat'])
del(g_und.vs['lon'])

### Ego-nets and communities

Suppose we want to look at node "near" some seed node $v$. One common way to do this is to look at its **ego-net**, i.e. the subgraph consisting of node $v$ and all other nodes that can be reached from $v$ in $k$ hops or less, where $k$ is small, typically 1 or 2. 

Such subgraphs can become large quickly as we increase $k$. In the cells below, we look at ego-nets and compare with another approach to extract subgraph(s) around $v$ via clustering.

We consider the undirected airport graph. We compare ego-nets (1 and 2-hops subgraphs from a given node) with clusters obtained via graph clustering for some vertex $v$ (you can try other vertices).

Let's explore 'SCF' (not too high degree to allow for easy viz). 

In [ ]:
v = g_und.vs.find('SCF').index
g_und.vs['color'] = 'lightblue'
g_und.vs[v]['color'] = 'black' # make SCF black so we can easily spot it

Show its ego-net for k=1 

In [ ]:
sg = g_und.subgraph([i for i in g_und.neighborhood(v,order=1)])
print(sg.vcount(),'nodes',sg.ecount(),'edges')
ig.plot(sg,bbox=(0,0,300,300), vertex_size=8)

Show the 2-core of its 2-hops ego-net ... this is already quite large!

In [ ]:
sg = g_und.subgraph([i for i in g_und.neighborhood(v,order=2)])
sg.vs()['core'] = sg.coreness()
sg.delete_vertices([v for v in sg.vs if v['core']<2])
print(sg.vcount(),'nodes',sg.ecount(),'edges')
ig.plot(sg,bbox=(0,0,300,300), vertex_size=8, edge_color='lightgrey')

Now compare this to using Louvain clustering and show the 2-core of the cluster containing the selected vertex



In [ ]:
m = g_und.vs[v]['ml']
sg = g_und.subgraph([i for i in range(g_und.vcount()) if ml.membership[i]==m])
sg.vs()['core'] = sg.coreness()
sg.delete_vertices([v for v in sg.vs if v['core']<2])
print(sg.vcount(),'nodes',sg.ecount(),'edges')
ig.plot(sg,bbox=(0,0,300,300), vertex_size=8, edge_color='lightgrey')

What happens if we prune nodes 3+ hops away?

In [ ]:
v = sg.vs.find(color='black')
ssg = sg.subgraph([i for i in sg.neighborhood(v,order=2)])
ssg.vs()['core'] = ssg.coreness()
ssg.delete_vertices([v for v in ssg.vs if v['core']<2])
print(ssg.vcount(),'nodes',ssg.ecount(),'edges')
ig.plot(ssg,bbox=(0,0,300,300), vertex_size=8, edge_color='lightgrey')

### Questions

#### 1. What are the most frequent states in each of the Louvain clusters (ml) we found?


#### 2. Find all airports of degree at least 10 that have all of their neighbours in their own Louvain cluster. 


### Possible Solutions

In [ ]:
## Louvain - most frequent states
for sg in ml.subgraphs():
    print('cluster:',sg.vcount(),'nodes, frequent states:',Counter(sg.vs['state']).most_common(5))
    
## store community and degree for each node for quick reference
g_und.vs['ml'] = ml.membership
g_und.vs['degree'] = g_und.degree()

## compute number of neighbours in same community for each node
for v in g_und.vs:
    if len(set([g_und.vs[w]['ml'] for w in g_und.neighborhood(v)])) == 1:
        if v['degree'] >= 10:
            print(v['name'],v['city'],v['state'],v['degree'])

In [ ]:
## clean-up
del(g_und.vs['ml'])
del(g_und.vs['degree'])

# To go further

More topics can be found in:
* book: https://www.ryerson.ca/mining-complex-networks
* notebooks: https://github.com/ftheberge/GraphMiningNotebooks
    
including:   
* more centrality measures
* clustering: overlapping clusters, outliers    
* degree assortativity
* vertex and graph embedding
* hypergraphs
* network robustness
* road networks


## 2.4 Bonus module - Community-based anomaly detection

### New dataset -- American College Football Graph

This is a nice, small graph for illustrating anomaly detection methods.

The graph consists of 115 US college football teams (nodes) playing games (edges).

Teams are part of 12 conferences (the 'communities'):
*   0 = Atlantic Coast
*   1 = Big East
*   2 = Big Ten
*   3 = Big Twelve
*   4 = Conference USA
*   5 = Independents
*   6 = Mid-American
*   7 = Mountain West
*   8 = Pacific Ten
*   9 = Southeastern
*  10 = Sun Belt
*  11 = Western Athletic

14 teams out of 115 appear as "anomalies", namely:
- the 5 teams in #5 conference (Independent) play teams in other conferences 
- the 7 teams in #10 conference (Sun Belt) are broken in 2 clumps 
- 2 teams from #11 conference play mainly with #10 conference

[REF]: "Community structure in social and biological networks", M. Girvan and M. E. J. Newman
PNAS June 11, 2002 99 (12) 7821-7826; https://doi.org/10.1073/pnas.122653799


In [ ]:
cfg = ig.Graph.Read_Ncol(datadir+'Football/football.edgelist',directed=False)
c = np.loadtxt(datadir+'Football/football.community',dtype='uint16',usecols=(0))
cfg.vs['community'] = [c[int(x['name'])] for x in cfg.vs]

Let's look at the College Football Graph and show communities in different colors, and anomalies as squares

In [ ]:
cfg.vs['shape'] = 'circle'
cfg.vs['anomaly'] = 0
pal = ig.ClusterColoringPalette(n=max(cfg.vs['community'])+1) 
cfg.vs['color'] = [pal.get(int(i)) for i in cfg.vs['community']]
for v in cfg.vs:
    if v['community'] in [5,10] or v['name'] in ['28','58']:
        v['shape']='square'
        v['anomaly']=1
ly = cfg.layout_fruchterman_reingold()
ig.plot(cfg, layout=ly, bbox=(0,0,500,300), vertex_size=6, edge_color='lightgray')


Find the participation coefficient, a high value indicative of outlier

In [ ]:
ec = cfg.community_ecg()
cfg.vs['ecg'] = ec.membership
for v in cfg.vs:
    l = list(Counter([cfg.vs[x]['ecg'] for x in cfg.neighbors(v)]).values()) ## neighbour's communities
    s = sum(l)
    v['pc'] = 1-sum([i**2/s**2 for i in l]) ## measures the homogeneity 
plt.rcParams['font.size'] = '14'
X = [v['pc'] for v in cfg.vs if v['anomaly']==0]
Y = [v['pc'] for v in cfg.vs if v['anomaly']==1]
plt.boxplot([X,Y],labels=['Regular','Outlier'],sym='.',whis=(0,100), widths=.5)
plt.ylabel('Participation coefficient',fontsize=14);

Use ECG: node strongly in a community has high ecg weights to neighbours

In [ ]:
cfg.es['ecg_weight'] = ec.W
for v in cfg.vs:
    v['ecg'] = np.mean([cfg.es[e]['ecg_weight'] for e in cfg.incident(v)])
X = [v['ecg'] for v in cfg.vs if v['anomaly']==0]
Y = [v['ecg'] for v in cfg.vs if v['anomaly']==1]
plt.boxplot([X,Y],labels=['Regular','Outlier'],sym='.',whis=(0,100), widths=.5)
plt.ylabel('ECG weights',fontsize=14);
